In [1]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf

import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
 

import getpass
from apikey import apikey 


In [2]:
def get_products():

    conn, cursor = SQLf.sql_db_functions.connect_sql()

    query = f"SELECT Brand_Prod_ID, Descript FROM Products ;" #LIMIT 5
    cursor.execute(query)

    # Fetch the rows
    rows = cursor.fetchall()

    df = pd.DataFrame(data=rows, columns=['Brand_Prod_ID','Descript'])

    SQLf.sql_db_functions.close_connection_db(conn=conn, cursor=cursor)

    return df


In [3]:
def get_related_links(Brand_Prod_ID):
    conn, cursor = SQLf.sql_db_functions.connect_sql()

    query = f"SELECT image_link FROM product_img WHERE Brand_id = {Brand_Prod_ID} ;"
    cursor.execute(query)
    # Fetch the rows
    image_links = cursor.fetchall()

    SQLf.sql_db_functions.close_connection_db(conn=conn, cursor=cursor)

    df_images = pd.DataFrame(data=image_links, columns=['image_link'])
    df_images.head()

    df_images['image_link'] = df_images['image_link'].apply(lambda x: x if 'https' in x else 'https:' +  x )

    return df_images.image_link

In [4]:
def generate_prompt(image_urls, description,prompt_instructions):
    # Construct the image part of the prompt
    images_text = ' '.join([f"{url}" for url in image_urls])
    # Combine with the description
    prompt_text = f"""
        {images_text} \n
        Description: {description} \n
        
        {prompt_instructions}
    """
    return prompt_text

In [5]:
def get_gpt_description(current_images, current_description):

    prompt_instructions = """

    The images are of the same shirt or top, use them to create a detailed description of it and its style.

    Take into account the following characteristics: 

    Fit, Sleeve style, Neckline, Material, Formality, Seasson, Colors, Texture, Transparency, Details and Embellishments, Shape,
    Length, Collar Style, Sleeve Style, Patterns, Patterns placement, Fluidity of fabric, Fabric weight, Pocket Presence, Pocket placement, 
    Pocket size, Breathability, Occasion Suitability, Lapel, etc.


    Use information from the image and the description.
    IGNORE BACKGROUND     
    ANSWER MUST BE AS SPECIFIC AS POSSIBLE BUT NOT COMPLEX
    WRITE IN A SINGLE PARAGRAPH
    NAME AND DESCRIBE THE COLORS USED
    """

    prompt_text = generate_prompt(current_images, current_description,prompt_instructions)


    os.environ['OPENAI_API_KEY'] = apikey

    # LLM
    model = ChatOpenAI(model="gpt-4o", temperature=0.7)

    # Prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a fashion specialist."),
            ("user", prompt_text),
        ]
    )

    chain = prompt | model

    response = chain.invoke({})
    return response.content



In [6]:
def get_gpt_form(current_images, current_description):

    prompt_instructions = """

    The images are of the same clothing peace, use them to fill the following format to list attributes of the clothing peace in image, separate them with comma:  

    *Type:  
    *Fit: (Slim fit, regular fit, oversized, tight, loose, etc)
    *Sleeve style: (Short sleeve, long sleeve, sleeveless, cap sleeve, etc)     
    *Neckline: (Crew neck, V-neck, scoop neck, etc)   
    *Material: (Cotton, linen, silk, polyester, etc)
    *Formality: (Casual, Business Casual, Formal, etc)
    *Seasson: (Winter, Spring, Summer, Autum)
    *Colors:  (percentage of each color in the clothing peace) 
    *Texture: (Smooth, Rough, Ribbed, Velvety, etc)
    *Transparency: (Opaque, Semi-transparent, Transparent, etc)
    *Details and Embellishments: (e.g, buttons, zippers, embroidery)
    *Shape: (e.g., boxy, fitted, flared) 
    *Length: (e.g., cropped, hip-length, tunic)
    *Collar Style: (e.g., crew neck, V-neck, polo, button-down)
    *Sleeve Style: (e.g., short, long, three-quarter, sleeveless, puffed)
    *Patterns: (e.g., stripes, floral, geometric, not applicable)
    *Patterns placement: (e.g., lower back, left sholder, right chest, not applicable) 
    *Fluidity of fabric:
    *Fabric weight: (Light, medium, or heavy)
    *Pocket Presence:(Yes,No)
    *Pocket placement:(e.g., lower back, left sholder, right chest, not applicable)
    *Pocket size:(small, medium, big, not applicable)
    *Breathability: (Low, Medium, High)
    *Occasion Suitability: (Casual, formal, business casual, etc)
    *Lapel: (Notch, Peak, Shawl Satin, Slim, Wide, Contrasting, etc)


    Use information from the image and the description.
    IGNORE BACKGROUND
    ONLY ANSWER THE FORMAT  
    DONT USE THE BRAND NAME IN ANY DESCRIPTION  
    IN EACH DESCRIPTION - CHARACTERISTIC SHOULD BE ITS OWN DESCRIPTION
    YOU CAN USE MULTIPLE DESCRIPTIONS IN A SINGLE ATTRIBUTE  
    DONT USE MULTICOLOR OR Multicolor, NAME THE SPECIFIC COLORS  
    COLORS SHOULD INCLUDE PRINT AND CLOTHE COLORS WITHOUT ANY DESCRIPTION OF THE PRINT ONLY MAIN COLORS
   
    """

    prompt_text = generate_prompt(current_images, current_description,prompt_instructions)


    os.environ['OPENAI_API_KEY'] = apikey

    # LLM
    model = ChatOpenAI(model="gpt-4o", temperature=0.7)

    # Prompt template
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a fashion specialist."),
            ("user", prompt_text),
        ]
    )

    chain = prompt | model

    response = chain.invoke({})
    return response.content



In [7]:
def load_form_to_db(current_product_ID,form, summary):

    
    conn, cursor = SQLf.sql_db_functions.connect_sql()

    query = """
    INSERT INTO product_characteristics (Brand_id, Detail, Summary) VALUES (%s, %s, %s)
    RETURNING Brand_id;
    """
    cursor.execute(query, (current_product_ID,form, summary))

    # Fetch the rows
    rows = cursor.fetchall()

    conn.commit()

    SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

    return rows

    


In [8]:
df = get_products()

In [9]:
df.head()

,Brand_Prod_ID,Descript
0,1,Il monogramma GG è riproposto in versioni e de...
1,2,Il monogramma GG è riproposto in versioni e de...
2,3,Ispirato allo spirito estivo e ai beach club d...
3,4,Ispirato allo spirito estivo e ai beach club d...
4,5,Ispirato allo spirito estivo e ai beach club d...


In [10]:
n_rows = len(df)

for current_product_ID in df.Brand_Prod_ID:

    current_image_links = get_related_links(current_product_ID)
    current_description = df[df.Brand_Prod_ID == current_product_ID]['Descript']

    cloth_summary = get_gpt_description(
        current_images=current_image_links,
        current_description=current_description
        )
    
    
    cloth_form = get_gpt_form(
        current_images=current_image_links,
        current_description=cloth_summary + '/n' + current_description
        )

    
    load_form_to_db(current_product_ID = current_product_ID,form=cloth_form, summary= cloth_summary)
    
    if current_product_ID % 10 == 0 | n_rows == 1:
        print(f"loaded {(current_product_ID-1)/n_rows}")